In [0]:
%sql
  DROP TABLE  IF EXISTS table_train;


In [0]:
%sql
create table table_train
using csv
OPTIONS(path '/FileStore/tables/landing/train.csv', header 'true')

In [0]:
%sql
select * from table_train limit 6

id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0
6,Female,24,1,33.0,0,< 1 Year,Yes,2630.0,160.0,176,0


###numero de linhas

In [0]:
%sql
select count(id) from table_train  



count(id)
381109


### Consultando  o ID para ver se temos eles duplos, comparando com o numero de linhas

In [0]:
%sql
SELECT DISTINCT  count(id)
FROM table_train;

count(id)
381109


## consultando se existe id duplo

In [0]:
%sql
SELECT DISTINCT  count(id)
FROM table_train;

count(id)
381109


###Consultando as colunas da tabela para ver se  tem valores null ou vazio

In [0]:
spark.sql("SELECT * FROM table_train where Age IS NULL OR GENDER IS NULL OR Driving_License IS NULL OR Region_Code IS NULL ").show()

+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+



In [0]:
spark.sql("SELECT * FROM table_train where Previously_Insured IS NULL OR Vehicle_Age IS NULL OR Vehicle_Damage IS NULL OR Annual_Premium IS NULL ").show()

+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+



In [0]:

spark.sql("SELECT * FROM table_train where Policy_Sales_Channel IS NULL OR Vintage IS NULL OR Vehicle_Damage IS NULL OR Response IS NULL").show()

+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+



In [0]:
%sql
SHOW COLUMNS FROM table_train

col_name
id
Gender
Age
Driving_License
Region_Code
Previously_Insured
Vehicle_Age
Vehicle_Damage
Annual_Premium
Policy_Sales_Channel


In [0]:
df_train = spark\
           .read.format('csv')\
           .option('InferSchema','true')\
           .option('header','true')\
           .csv('/FileStore/tables/landing/train.csv')

In [0]:
display(df_train.printSchema)

<bound method DataFrame.printSchema of DataFrame[id: int, Gender: string, Age: int, Driving_License: int, Region_Code: double, Previously_Insured: int, Vehicle_Age: string, Vehicle_Damage: string, Annual_Premium: double, Policy_Sales_Channel: double, Vintage: int, Response: int]>

## Mudando a coluna 'Driving_License' de 0: "No", 1: "Yes"

In [0]:
# Mudando a coluna 'Driving_License' de 0: "No", 1: "Yes"
df_train=df_train.withColumn('Driving_License',regexp_replace(df_train['Driving_License'],'1','Yes'))
df_train=df_train.withColumn('Driving_License',regexp_replace(df_train['Driving_License'],'0','No'))

## Mudando a coluna 'Vehicle_Age'

In [0]:
df_train=df_train.withColumn('Vehicle_Age',regexp_replace(df_train['Vehicle_Age'],'> 2 Years','2'))

df_train=df_train.withColumn('Vehicle_Age',regexp_replace(df_train['Vehicle_Age'],'< 1 Year','0'))

df_train=df_train.withColumn('Vehicle_Age',regexp_replace(df_train['Vehicle_Age'],'1-2 Year','1'))

## Mudando a coluna Vehilcle_Damage

In [0]:
df_train=df_train.withColumn('Vehicle_Damage',regexp_replace(df_train['Vehicle_Damage'],'No','0'))
df_train=df_train.withColumn('Vehicle_Damage',regexp_replace(df_train['Vehicle_Damage'],'Yes','1'))

## Coluna  Gender transformando 'Female': 0, 'Male': 1

In [0]:
df_train=df_train.withColumn('Gender',regexp_replace(df_train['Gender'],'Female','0'))
df_train=df_train.withColumn('Gender',regexp_replace(df_train['Gender'],'Male','1'))

In [0]:
display(df_train.show(5))

+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
|  1|     1| 44|            Yes|       28.0|                 0|          2|             1|       40454.0|                26.0|    217|       1|
|  2|     1| 76|            Yes|        3.0|                 0|          1|             0|       33536.0|                26.0|    183|       0|
|  3|     1| 47|            Yes|       28.0|                 0|          2|             1|       38294.0|                26.0|     27|       1|
|  4|     1| 21|            Yes|       11.0|                 1|          0|             0|       28619.0|               152.0|    203|  

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, DoubleType, TimestampType
from pyspark.sql.functions import col , regexp_replace
 

### Transformando as Colunas  "Vehicle_Age", 'Vehicle_Damage' e "Gender" em Integer

In [0]:
df_train = df_train.withColumn("Vehicle_Age",col("Vehicle_Age").cast(IntegerType()))

In [0]:
df_train = df_train.withColumn('Vehicle_Damage',col('Vehicle_Damage').cast(IntegerType()))

In [0]:
df_train = df_train.withColumn("Gender",col("Gender").cast(IntegerType()))
df_train = df_train.withColumn('Driving_License',col('Driving_License').cast(StringType()))

In [0]:
display(df_train.printSchema)

<bound method DataFrame.printSchema of DataFrame[id: int, Gender: int, Age: int, Driving_License: string, Region_Code: double, Previously_Insured: int, Vehicle_Age: int, Vehicle_Damage: int, Annual_Premium: double, Policy_Sales_Channel: double, Vintage: int, Response: int]>

In [0]:
display(df_train)

id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
1,1,44,Yes,28.0,0,2,1,40454.0,26.0,217,1
2,1,76,Yes,3.0,0,1,0,33536.0,26.0,183,0
3,1,47,Yes,28.0,0,2,1,38294.0,26.0,27,1
4,1,21,Yes,11.0,1,0,0,28619.0,152.0,203,0
5,0,29,Yes,41.0,1,0,0,27496.0,152.0,39,0
6,0,24,Yes,33.0,0,0,1,2630.0,160.0,176,0
7,1,23,Yes,11.0,0,0,1,23367.0,152.0,249,0
8,0,56,Yes,28.0,0,1,1,32031.0,26.0,72,1
9,0,24,Yes,3.0,1,0,0,27619.0,152.0,28,0
10,0,32,Yes,6.0,1,0,0,28771.0,152.0,80,0
